### Test Notebook for Marketing Dashboard: FB Page Analysis Skooldio

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import re

In [ ]:
# Please change name of dataset to sample before run code
df = pd.read_csv('sample.csv')
df

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
# Drop unnecessary columns
df.drop(columns=['Sticker taps', 'Content type', 'Replies', 'Results', 'Cost per result'], inplace=True)
df.head()

In [ ]:
# Change to Datetime format to be able to analyze data trends
df['Post time'] = pd.to_datetime(df['Post time'])

In [ ]:
# Exclude change cover post
df = df.loc[df['Caption'] != 'Skooldio updated their cover photo.'].reset_index(drop=True)

In [ ]:
# Inspect correlation between variables
data_temp = df.corr(method='kendall')
sns.heatmap(data_temp, cmap='Blues', annot=True)

In [ ]:
# Prepare available columns to get tags in caption
df['tags'] = np.array([np.nan]*df.shape[0])
df

In [ ]:
# Extract all "English" Tags to tags column
pattern = re.compile('#[a-zA-Z0-9]+[\w$]')

for index, row in df.iterrows():
    result = pattern.findall(df['Caption'][index])
    lower_result = [e.lower() for e in result]
    df['tags'][index] = lower_result

df

In [ ]:
from collections import Counter
from itertools import chain
pd.Series(Counter(chain(*df.tags))).sort_values(ascending=False)

In [ ]:
# Check outliers
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()

df_box = df[['Reach', 'Likes and reactions', 'Post Clicks', 'Comments', 'Shares']]
df_box_scaled = pd.DataFrame(min_max_scaler.fit_transform(df_box))
df_box_scaled.columns = ['Reach', 'Likes and reactions', 'Post Clicks', 'Comments', 'Shares']
df_box_scaled

sns.boxplot(data=df_box_scaled, palette='Paired')
plt.xticks(rotation=30)
plt.ylabel('Normalized Values')

In [ ]:
df.describe()

In [ ]:
df['Reach'].plot(x='Post time')
df['Likes and reactions'].plot(x='Post time', color='green', alpha=0.7)

In [ ]:
df.sort_values(by='Reach', ascending=False)

In [ ]:
# Remove Outliers (Boost / Ads Post)
# Outliers 
# df[~(np.abs(df.Reach-df.Reach.mean()) <= (3*df.Reach.std()))]
df = df[(np.abs(df.Reach-df.Reach.mean()) <= (3*df.Reach.std()))]

In [ ]:
df.plot(kind='line', x='Post time', y='Reach')
df.plot(kind='line', x='Post time', y='Likes and reactions', color='green', alpha=0.7)
plt.show()

In [ ]:
df.sort_values(by='Reach', ascending=False)

In [ ]:
df.insert(loc=0, column='postID', value=range(1, len(df) + 1))
df = df.reset_index(drop=True)
df

In [ ]:
df['postID'] = df['postID'].astype(str)

In [ ]:
df.loc[df.isnull().any(axis=1)]['Caption']

In [ ]:
for index, row in df.iterrows():
    print(list(row))
    break

In [ ]:
for index, row in df.iterrows():
    print(len(tuple(row)))
    break

In [87]:
import numpy as np
import pandas as pd
import re
import sqlite3

# what to modify in this script
# columns name is fixed, if it change --> rename columns

# import data from csv file
df1 = pd.read_csv('sample.csv')

# pre-processing
df1.drop(columns=['Sticker taps', 'Content type', 'Replies', 'Results', 'Cost per result'], inplace=True)
# df1['Post time'] = pd.to_datetime(df1['Post time'])
df1 = df1.loc[df1['Caption'] != 'Skooldio updated their cover photo.'].reset_index(drop=True)
df1.insert(loc=0, column='postID', value=range(1, len(df1) + 1))
df1 = df1.reset_index(drop=True)
df1['postID'] = df1['postID'].astype(str) # ERROR

# create a DataFrame for each table in a database file
dict_df2 = {'tagID':[], 'campaigns':[]}
dict_df3 = {'pk_postsAndTags':[], 'postID':[], 'tagID':[]}
# dict_df4 = {'tagID':[], 'courseName':[]}
# dict_df5 = {'customerID':[], ...}

# extract tags from Caption (exclude Thai language)
# indicate regular expression
pattern = re.compile('#[a-zA-Z0-9]+[\w$]') # indicate REGEX for retrieve tags
# Extract tags for each post into a new table
for index, row in df1.iterrows(): # find tags for each post (each row)
    result = pattern.findall(df1['Caption'][index]) # get tags from a comment column and get into a list
    lower_result = [e.lower() for e in result] # lowering case tags for make it consistent
    lower_result_remove_duplicate = list(set(lower_result)) # remove duplicate tags for each post
    if len(lower_result_remove_duplicate) != 0: # if post contain any tag
        # lower_result is a list containing tags for each post; [#tag1, #tag2, #tag3, ...]
        for e in lower_result_remove_duplicate: # for each tag in a post
            if e not in dict_df2['campaigns']: # if tag is not in record before, record it as a new one
                tag_id = len(dict_df2['tagID'])+1
                dict_df2['tagID'].append(str(tag_id))
                dict_df2['campaigns'].append(e)

            # define elements before inserting it to be the data
            postid = df1['postID'][index]
            i = dict_df2['campaigns'].index(e)
            tagid = dict_df2['tagID'][i]
            temp_tuple = str(tuple([postid, tagid]))
            # insert elements to be the data
            dict_df3['pk_postsAndTags'].append(temp_tuple)
            dict_df3['postID'].append(postid)
            dict_df3['tagID'].append(tagid)

df2 = pd.DataFrame(dict_df2)
df3 = pd.DataFrame(dict_df3)

In [91]:
lower_result_remove_duplicate

['#UpskillWithSkooldio']

In [55]:
df1.iloc[1]['Caption'] # postID == 2 has no tag in caption
# there're some post not containing any tag, leading to being unable to analysis

'👨\u200d⚖️ ออกแบบเว็บไซต์ยังไงให้ถูกต้องตามหลัก PDPA⁉️\n.\n✍️ ก่อนอื่นเรามารู้จักกับกฎหมาย PDPA กันคร่าวๆ ก่อน 🤩 PDPA ย่อมาจาก Personal Data Protection Act หรือพ.ร.บ.คุ้มครองข้อมูลส่วนบุคคล ถูกกำหนดขึ้นเพื่อคุ้มครองข้อมูลส่วนบุคคลของผู้บริโภคจากคน บริษัท หรือองค์กรต่างๆ ข้อมูลดังกล่าวอาจเป็นชื่อ ที่อยู่ ข้อมูลการศึกษา รวมถึงข้อมูลบนอินเทอร์เน็ตเช่น Username Password, Cookies IP Address หรือ GPS Location\n.\n🎨 ดังนั้นหน้าที่ของ UX Designer คือการปรับตัวตามสถานการณ์ สิ่งที่สำคัญสำหรับการออกแบบคือการแจ้งให้ผู้ใช้ทราบว่าจะมีการจัดเก็บข้อมูล พร้อมทั้งถามความยินยอม และแจ้งรายละเอียดต่างๆ ให้ชัดเจน 🙌🏻เราได้รวบรวมตัวอย่างการออกแบบเว็บไซต์ที่สอดคล้องกับ PDPA ไว้ 4 ตัวอย่างง่ายๆ ลองไปดูกันเลย!'

In [89]:
condition = df3['pk_postsAndTags'].duplicated()
df3[condition]

,pk_postsAndTags,postID,tagID


In [82]:
for index, row in df3.iterrows():
    print(row)
    break

pk_postsAndTags    ('1', '1')
postID                      1
tagID                       1
Name: 0, dtype: object
